# Sensitivity analysis of population parameters

In [ ]:
import testing_and_analysis_functions as analysis_func
import poisson_processes as pp
import pointwise_correlation as pc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import Louvain
plt.rcParams['figure.figsize']=[20,10]

verbose=False
individuals_within_a_population_have_same_incidence=True
copy_results_out_of_temp_dir=False
reclustering='greedy Louvain'

length = 10000
number_of_populations=4
size=25
lag=10
event_prob=0.05
noise_prob=0.2
interpret_as_max_values=False

if interpret_as_max_values:
    metaparams=analysis_func.create_random_metaparams(length=length,number_of_populations=number_of_populations,
                                                      max_size=size,max_lag=lag,max_event_prob=event_prob,max_noise_prob=noise_prob)
else:
    metaparams=analysis_func.create_fixed_metaparams(length=length,number_of_populations=number_of_populations,
                                                     size=size,lag=lag,event_prob=event_prob,noise_prob=noise_prob)

population_params=analysis_func.directly_initialise_multiple_populations(length=length,metaparams=metaparams,
                                                       use_fixed_means=individuals_within_a_population_have_same_incidence)

#Copy parameters to csv file in TEMP directory
pd.DataFrame.from_dict(metaparams,orient='index').to_csv("{0}\meta_params.csv".format(pc.TEMP_DIR))
df=pd.DataFrame.from_dict(population_params,orient='index')
df['prior process beta'] = df['prior poisson process'].apply(lambda x: None if type(x)==float else x.betas)
df['mean lag list']=df['lag parameters'].apply(lambda x: None if type(x)==float else x['mus'])
df=df.drop(['prior poisson process','lag parameters'],axis=1)
df.to_csv("{0}\population_parameters.csv".format(pc.TEMP_DIR))
    
    
#Intialise time series objects within their populations    
mpp = pp.mixed_poisson_populations(length,population_params,verbose=verbose)
mpp.display(stats=True)

# mix populations and extract the full array of time_series_objects
ts_obj1=mpp.randomly_mix_populations(list(metaparams.keys()))
number=len(ts_obj1)   # total number of all time series in all populations        
ts_obj2=[]

# compare v1 and v2 sigma measures and store correlations
td,df=analysis_func.compare_inferred_and_known_means(xs=[length],number=number,ts_matrices=[ts_obj1,ts_obj2],
                                                    reclustering='greedy Louvain')
plt.show()
display(df)

# copy results out of pc.TEMP_DIR directory into given directory
if copy_results_out_of_temp_dir:
    analysis_func.copy_from_temp(dest_root_dir="{0}/Multiple_population_correlations".format(pc.RESULTS_DIR))

if reclustering=='greedy Louvain':
    v1_df,v2_df=analysis_func.load_results_to_dfs(pc.TEMP_DIR)
    scores_v1={'version 1':Louvain.make_partition_and_score(v1_df,test_random_graph=False,pass_weights=True,verbose=False)}
    display(pd.DataFrame.from_dict(scores_v1,orient='index'))
    scores_v2={'version 2':Louvain.make_partition_and_score(v2_df,test_random_graph=False,pass_weights=True,verbose=False)}
    display(pd.DataFrame.from_dict(scores_v2,orient='index'))
    


Initialising a random population None, size 1, with betas [20.0]
Betas for population None are [20.0]
Initialising a random population None, size 1, with betas [20.0]
Betas for population None are [20.0]
Initialising a random population None, size 1, with betas [20.0]
Betas for population None are [20.0]
Initialising a random population None, size 1, with betas [20.0]
Betas for population None are [20.0]
Initialising for key Owen_noise
Expected 25 beta values, but only 1 values passed.  Initialising all time series with beta parameter 5.0
Initialising a random population Owen_noise, size 25, with betas [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
Betas for population Owen_noise are [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
Elapsed time: 0.31406402587890625
Initialising for key Owen
Creating lagging time series population ba

In [3]:
import pointwise_correlation as pc
import pandas as pd
import Louvain
import testing_and_analysis_functions as analysis_func
v1_df,v2_df=analysis_func.load_results_to_dfs(pc.TEMP_DIR)
#v1_df=pd.read_csv("{0}/sigma_v1_correlations.csv".format(pc.TEMP_DIR))
print(v1_df.head(10))
#scores_v1=Louvain.make_partition_and_score(v1_df,test_random_graph=False,pass_weights=True,verbose=False)
#display(pd.DataFrame.from_dict({'v1':scores_v1},orient='index'))
#scores_v2=Louvain.make_partition_and_score(v2_df,test_random_graph=False,pass_weights=True,verbose=False)
#display(pd.DataFrame.from_dict({'v2':scores_v2},orient='index'))

df_results=v1_df
names=list(set(df_results['name1']).union(set(df_results['name2'])))
print(names)
#nodes=list(set(df_results['object1']).union(set(df_results['object2'])))
#print(df_results.head(40))
#print([df_results.loc[i,'object1'] for i in df_results.index[:10]])
#name_of_node={df_results.loc[i,'object1']:df_results.loc[i,'name1'] for i in df_results.index}
#id_test=df_results.loc[0,'object1']
#print(id_test)
#print(type(id_test))

name_of_node={}
for i in df_results.index:
    name_of_node[df_results.loc[i,'object2']]=df_results.loc[i,'name2']
    
print(name_of_node)

   p-value  Z-score name1 name2        object1        object2
0    0.434   -0.165  Jill   Flo  2289691956056  2289691761128
1    0.306   -0.507  Jill  Owen  2289691956056  2289691955272
2    0.597    0.246  Jill  Jill  2289691956056  2289691956224
3    0.574    0.186  Jill  Jill  2289691956056  2289691955888
4    0.735    0.628  Jill   Tom  2289691956056  2289691762192
5    0.493   -0.019  Jill  Owen  2289691956056  2289691955384
6    0.441   -0.148  Jill  Jill  2289691956056  2289691956112
7    0.683    0.476  Jill   Flo  2289691956056  2289691761184
8    0.297   -0.532  Jill   Flo  2289691956056  2289691761352
9    0.122   -1.163  Jill   Tom  2289691956056  2289691761856
['Tom', 'Flo', 'Jill', 'Owen']
{2289691761128: 'Flo', 2289691955272: 'Owen', 2289691956224: 'Jill', 2289691955888: 'Jill', 2289691762192: 'Tom', 2289691955384: 'Owen', 2289691956112: 'Jill', 2289691761184: 'Flo', 2289691761352: 'Flo', 2289691761856: 'Tom', 2289691956168: 'Jill', 2289691762080: 'Tom', 2289691955496: '